# Ising Model ML Exploration

This notebook demonstrates how to load Ising model data, train a VDNN model, and evaluate its performance using the modular codebase.

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from ising.training.train_dnn import train_dnn

## Load and Prepare Ising Dataset

Load the Ising dataset, split into training and test sets, and preprocess as needed.

In [ ]:
# Example: Replace with actual data loading from ising.data.data_loader
# For demonstration, generate random data
n_samples = 1000
n_features = 40*40
X = np.random.rand(n_samples, n_features)
y = np.random.randint(0, 2, size=(n_samples, 1))

# Split into train and test sets
split = int(0.8 * n_samples)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

## Train VDNN Model

Train the VDNN model using the training data.

In [ ]:
# Train the VDNN model
model, losses = train_dnn(X_train, y_train, depth=4, epochs=10, lr=1e-3, batch_size=128)

## Plot Training Loss Curve

Visualize the training loss over epochs.

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(losses, marker='o')
plt.title('Training Loss Curve')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)
plt.show()

## Evaluate Model Performance

Use the trained model to make predictions on the test set and compute accuracy.

In [ ]:
# Evaluate on test set
model.eval()
with torch.no_grad():
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_pred = model(X_test_tensor).cpu().numpy()
    y_pred_label = (y_pred > 0.5).astype(int)
    accuracy = np.mean(y_pred_label == y_test)
print(f"Test Accuracy: {accuracy:.4f}")

In [ ]:
from ising.plotting.plotting import plot_adversarial_results

In [ ]:
# Example usage for G=0
# plot_adversarial_results(df_adversarial_G_0, df_ordinary_G_0, G_0_adversarial_scatter_plot_info_dict, G_label='0')

# Example usage for G=0.1
# plot_adversarial_results(df_adversarial_G_0_1, df_ordinary_G_0_1, G_0_1_adversarial_scatter_plot_info_dict, G_label='0_1')

# Example usage for G=0.5
# plot_adversarial_results(df_adversarial_G_0_5, df_ordinary_G_0_5, G_0_5_adversarial_scatter_plot_info_dict, G_label='0_5')

In [ ]:
import h5py
import os

# Example: Load G0_data.h5py from src/ising/data
h5_path = os.path.join(os.path.dirname(os.path.dirname(__file__)), 'data', 'G0_data.h5py')
with h5py.File(h5_path, 'r') as f:
    # Example: load dataset 'Ts' and print shape
    Ts = f['Ts'][:]
    print('Loaded Ts shape:', Ts.shape)
    # Add more data loading as needed

In [ ]:
import h5py
import numpy as np
import torch
import matplotlib.pyplot as plt

# Load data from src/ising/data
base_data_path = os.path.join(os.path.dirname(os.path.dirname(__file__)), 'data')
f5 = h5py.File(os.path.join(base_data_path, 'G0_data.h5py'), 'r')
f5_new = h5py.File(os.path.join(base_data_path, 'Gp1_data.h5py'), 'r')
f5_new_2 = h5py.File(os.path.join(base_data_path, 'Gp5_data.h5py'), 'r')
f5_new_3 = h5py.File(os.path.join(base_data_path, 'G1_J1_data.h5py'), 'r')
f5_new_4 = h5py.File(os.path.join(base_data_path, 'G1_Jp01_data.h5py'), 'r')

Ts_G_0 = f5['Ts'][:]
Targs = np.arange(61)
Targs_new = np.arange(80)
xiBs = f5['xiBs'][:]
factor = 0.30
cut = np.amax(xiBs) * factor
labels_tmp = np.heaviside(xiBs[Targs] - cut, 1.0)
T0, T1 = Ts_G_0[np.where(labels_tmp == 1.0)[0][[0, -1]]]

# Visualize labels
plt.figure(dpi=150)
plt.plot(Ts_G_0[8:-9], labels_tmp[8:-9], '.-')
plt.xticks(Ts_G_0[8:-9], rotation=90)
plt.tight_layout()
plt.grid(True)
plt.show()

# Prepare training and validation data
data_train = f5['arrs'][Targs, :8000, :]
data_validate = f5['arrs'][Targs, 8000:, :]
data_train = data_train.reshape(Targs.shape[0] * 8000, 1600)

# Additional validation sets for other G values
data_validate_new = f5_new['arrs'][Targs, 8000:, :]
data_validate_new2 = f5_new_2['arrs'][Targs, 8000:, :]
data_validate_new3 = f5_new_3['arrs'][Targs_new, 8000:, :]
data_validate_new4 = f5_new_4['arrs'][Targs, 8000:, :]

labels_train = np.repeat(labels_tmp, 8000)
labels_validate = np.repeat(labels_tmp, 2000)

data_train = torch.tensor(data_train)
labels_train = torch.tensor(labels_train, requires_grad=False)
labels_validate = torch.tensor(labels_validate, requires_grad=False)